In [36]:
import numpy as np
import random

In [69]:
class Network(object):
    
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y,1) for y in sizes[1:]]
        self.weights = [np.random.randn(y,x) for x,y in zip(sizes[:-1], sizes[1:])]
        
    def feedforward(self, a):
        for b,w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        if test_data:
            n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] 
                            for k in xrange(0,n,mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print "Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test)
            else:
                print "Epoch {0}".format(j)
                
    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x,y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+ndb for nb, ndb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+ndw for nw, ndw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]
        
    def backprop(self, x, y):

        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        activation = x
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        return (output_activations-y)


In [21]:
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

In [10]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

In [22]:
import pandas as pd

In [24]:
data = pd.read_csv("train.csv")

In [25]:
data.shape

(42000, 785)

In [26]:
y = data.ix[:,0]
X = data.ix[:,1:]

In [28]:
from sklearn.cross_validation import train_test_split

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=0, stratify=y)

In [30]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.1, random_state=0, stratify=y_train)

In [81]:
training_data = [(x.reshape(784,1), vectorized_result(y)) for x,y in zip(X_train.values, y_train.values)]
test_data = [(x.reshape(784,1), y) for x,y in zip(X_test.values, y_test.values)]

In [98]:
net = Network([784, 50, 10])

In [99]:
net.SGD(training_data, 30, 10, 0.5, test_data = test_data)

Epoch 0: 1914 / 4199
Epoch 1: 2059 / 4199
Epoch 2: 2027 / 4199
Epoch 3: 2185 / 4199
Epoch 4: 2016 / 4199
Epoch 5: 2074 / 4199
Epoch 6: 2144 / 4199
Epoch 7: 2239 / 4199
Epoch 8: 2349 / 4199
Epoch 9: 2248 / 4199
Epoch 10: 2500 / 4199
Epoch 11: 2535 / 4199
Epoch 12: 2589 / 4199
Epoch 13: 2317 / 4199
Epoch 14: 2310 / 4199
Epoch 15: 2732 / 4199
Epoch 16: 2596 / 4199
Epoch 17: 2247 / 4199
Epoch 18: 2160 / 4199
Epoch 19: 2457 / 4199
Epoch 20: 2707 / 4199
Epoch 21: 1967 / 4199
Epoch 22: 2262 / 4199
Epoch 23: 2206 / 4199
Epoch 24: 2205 / 4199
Epoch 25: 2624 / 4199
Epoch 26: 2502 / 4199
Epoch 27: 2649 / 4199
Epoch 28: 2876 / 4199
Epoch 29: 2868 / 4199


In [77]:
def vectorized_result(y):
    v = np.zeros((10,1))
    v[y] = 1.0
    return v